마이크로소프트의 SSL github repository를 활용하여 실험을 수행하였음.  
https://github.com/microsoft/Semi-supervised-learning

# 0. Intro

이번 실습은 Semi-supervised-learning 기법들을 직접 사용해 보고, 다른 방법론 사이의 비교를 수행해 보는 것을 목적으로 한다. 실습에서 활용하는 코드는 위에 첨부한 마이크로소프트의 SSL github repository로 다양한 SSL 알고리즘 들을 제공한다. 여기서 특히 Holistic methods에 집중하여 MixUp 방법론의 개선 연구의 흐름을 따라 MixMatch, ReMixMatch, FixMatch, FlexMatch, SimMatch에 대해 실험을 수행해 보고자 한다.  

전체적인 실습의 구성은 위 다섯 가지의 방법론에 대한 간단한 이론적 소개 후, 순서대로 동일한 데이터에 대해 실험을 수행한다. 이론 설명은 고려대학교 산업경영공학과 강필성 교수님 [IME654]Business-Analytics 수업자료와 DSBA Lab Seminar 및 PYSR 자료를 참고하여 재 구성 한 것이다.  

실험에는 방법론의 일반화 성능을 조금 더 강하게 살펴 보기 위해, 일반적으로 많이 사용되는 image domain이 아니라 NLP domain의 data를 활용하여 실험을 수행하였다.  

추가로 실험의 결과 리포팅의 경우 본 방법론들이 각각 시간이 많이 소요 되기 때문에, 따로 terminal에서 tmux를 통해 실험을 수행하고 노트북 파일에는 코드와 결과만 이미지로 첨부하였다.


https://sustaining-starflower-aff.notion.site/2022-2-0e068bff3023401fa9fa13e96c0269d7  
(2022 고려대학교 산업경영공학과 강필성 교수님 [IME654]Business-Analytics 수업자료)  

https://www.youtube.com/watch?v=nSJP7bn2D1U  
(DSBA Lab Seminar 2020, 이정훈)
 
https://youtu.be/NoPkSeKUER4  
(DSBA Lab PYSR 2022, 고유경)


# 1. Backgorund
## 1.1. 이론
### 1.1.0 Stochastic Data Augmentation
 <p align="center">   <image src="./images/2022-12-27-15-49-33.png" width="60%"/>   </p>

### 1.1.1 MixMatch
 <p align="center">   <image src="./images/2022-12-27-15-50-10.png" width="60%"/> <figcaption align="center">DSBA Lab Seminar 2020, 이정훈  </figcaption>  </p>
 
 
  기존의 MixUp 방법론의 경우 Unlabeled data만을 활용하여 MixUp을 수행하였기 때문에, 보다 정확한 정보를 줄 수 있는 labeled data를 MixUp에 함께 활용하기 위해 제안된 방법론이다.  

Labeled data의 경우 사전에 정의한 image Augmentation 기법 중 하나를 임의로 적용하고, unlabeled data의 경우는 hyperparameter로 정의 되는 K번 만큼의 Stochastic Data Augmentation을 수행한다. 두 데이터 셋의 augmentation 결과를 우선 함께 shuffle 한 후 각각의 data 수 만큼을 MixUp한다. 여기서 다음과 같이 Max()를 거쳐 얻어진 λ’ 은 항상 0.5보다 크기 때문에, 항상 앞쪽에 유사한 결과를 얻게 된다.
 <p align="center">   <image src="./images/2022-12-27-15-26-01.png" width="20%"/>  </p>

그렇게 Augmented data가 준비되면 다음으로는 Labeled는 정답에 가깝게하는 Cross Entropy Loss와 Unlabeled는 guessed label에 가깝게하는 L2 Loss 학습을 수행하게 된다.
 <p align="center">   <image src="./images/2022-12-27-15-28-31.png" width="25%"/>   </p>



### 1.1.2 ReMixMatch
MixMatch 방법론에서 다음의 두가지 목적을 가지고 개선된 모델이다.
1. Unlabeled 데이터의 전체적인  예측  분포가 Labeled 데이터의 분포와  비슷하도록 하고
2. Strong Augmentation을  적용하되  Guessed Label은  Weak Augmentation을  이용해서  구하자 

각각의 목표는 다음 그림에 나타난 두가지 모듈 Distribution Alignment, Augmentation Anchoring을 통해 구현된다.
 <p align="center">   <image src="./images/2022-12-27-15-57-41.png" width="60%"/>   </p>

- Distribution Alignment
  - Maximize Mutual Imformation
    :위 식은 확률변수  𝑥, 𝑦가  독립이  아니라면  더 높은값을 가짐
     <p align="center">   <image src="./images/2022-12-27-16-00-27.png" width="60%"/>   </p>
     식에서 앞쪽의 term: 예측결과의 분포가 Labeled data의 prior 분포와 가깝게! 현재는 균등한 분포일때의 식으로 다음의 식을 통해 현재 분포에 알맞게 조율된다.
      <p align="center">   <image src="./images/2022-12-27-16-03-10.png" width="25%"/>   </p>
     뒤쪽의 term: 좋은 모델은 confidence가 커야
-  Augmentation Anchoring  
    : weak augmentation만 수행한 경우는 데이터의 다양성을 충분히 확보할 수 없고, strong augmentation을 수행한 경우는 데이터의 정보가 많이 훼손 되어 guessed label을 신뢰할 수 없다.   
    이런 상황에서 strong augmentation을 수행하되, pseudo label의 경우는 같은 원 데이터로 부터 얻은 weak augmentaion 결과로 부터 얻겠다는 의미이다.
설명한 부분이외에는 기존의 MixMatch와 동일하며 최종적으로 다음의 Loss를 통해 학습이 수행된다.
 <p align="center">   <image src="./images/2022-12-27-16-16-32.png" width="60%"/>   </p>
    앞의 두 term은 각각 labeled, unlabeled data에 대한 loss이다.
     

### 1.1.3 FixMatch
이전 까지 소개한 방식이 너무 복잡하기 때문에 확실한 Weak Augmentation의 예측  결과는 바로  Strong Augmentation의 Pseudo-label로 사용하자는 접근으로 weak augmentation에 대해 anchoring 등 과정을 거치는 것이 아니라, 만약 weak augmentation의 예측 confidence가 어느 수준 이상으로 확실 하다면 그대로 strong augmentation의 pseudo label로 사용하겠다는 의미이다.  

기본적인 과정은 MixMatch와 동일하고 차이가 나는 부분만을 수식과 함께 정리하면 다음과 같다.
- Supervised Learning
  - labeled data에 대해서는 weak augmentation을 수행한 후 모델을 통해 예측을 수행하고 Supervised cross-entropy 학습이 수행된다.

   <p align="center">   <image src="./images/2022-12-27-16-22-57.png" width="23%"/>   </p>
- Unsupervised Learnin
  - 정답이 없는 경우 confidence가 넘는 weak aug.의 예측 결과가 Pseudo-label 역할을 수행
  Threshold를  넘으면  Pseudo-label 이용하고, Threshold를  넘지  않으면  loss를  계산하지 않는다. 결과적으로 학습  초기에는  대부분  threshold를 넘지  않아  supervised loss만  계산되어 별도의 장치 없이 자동으로 초반과 후반에 Labeled data와 unlabeled data의 반영 비율 조절 된다.


    <p align="center">   <image src="./images/2022-12-27-16-24-03.png" width="40%"/>   </p>

### 1.1.4 FlexMatch
다음 그림에서 확인할 수 있듯이 분류 task 수행시에 범주마다 난이도가 달르기 때문에 각 범주의 난이도에 따라 서로 다른 threshold를 사용하는 것이다.
 <p align="center">   <image src="./images/2022-12-27-16-25-59.png" width="60%"/>  <figcaption align="center">DSBA Lab PYSR 2022, 고유경  </figcaption>   </p>

위 과정은 Curriculum Pseudo Labeling을  통해  각  Class의  난이도에  따른  Confidence Threshold를  시점마다  다르게하는 것으로 구현되었으며, 전체적은 과정은 다음과 같다.
 <p align="center">   <image src="./images/2022-12-27-16-27-50.png" width="60%"/>   </p>

본 과정은 크게 두 가지로 나뉘는데 Learning Effect 측정, Learning Effect 정규화  및 Threshold 결정이 그것이다.

- Learning Effect 측정
  - 현재 Threshold 넘는 개수가 몇 개냐!?
  <p align="center">   <image src="./images/2022-12-27-16-29-31.png" width="40%"/>   </p>
- Learning Effect 정규화  및 Threshold 결정
  - 비율지표로 잘 맞출수록 1에 가까워져 해당 범주의 threshold는 τ에 수렴
  <p align="center">   <image src="./images/2022-12-27-16-30-05.png" width="40%"/>   </p>

여기에 추가로 모든  Class의  threshold가  0에서  시작해서  점진적으로  증가하게 하는 Threshold Warm-Up과
 <p align="center">   <image src="./images/2022-12-27-16-31-39.png" width="30%"/>   </p>
학습 초기의 변동성을 완화하고 β의 크기 조절을 통해 속도를 조절하는 Non-Linear Mapping이 적용된다.
 <p align="center">   <image src="./images/2022-12-27-16-32-05.png" width="30%"/>   </p>

Loss는 다음과 같다.
 <p align="center">   <image src="./images/2022-12-27-16-32-41.png" width="60%"/>   </p>



### 1.1.5 SimMatch
- Unlabeled Data에 대한 추가적인 학습 정보가 도입될 필요
- 학습 과정 중 저장되는 Memory Buffer를 이용하자
- 두 정보 간 차이를 최소화하도록 Loss 제안
  - Unlabeled의 Prediction Distribution vs Memory Buffer를 Prediction Distribution으로 변환
  - Unlabeled의 Prediction Distribution을 Representation으로 변환 vs Memory Buffer
- Class Center : Encoder를 통과한 후 Classification Head를 통과한 벡터(의미적 정보를 보존한 벡터, semantic vector)
- Labeled Embeddings : Encoder를 통과한 후 학습되는 MLP Layer를 통과한 벡터(labeled data의 정보를 저장하는 Memory Buffer의 역할)
 <p align="center">   <image src="./images/2022-12-27-17-06-23.png" width="60%"/>   </p>

 <p align="center">   <image src="./images/2022-12-27-17-06-45.png" width="60%"/>   </p>

## 1.2. repository 코드 활용 개요

### 1.2.1 config 정의를 통해 실험에 활용할 hyperparameter 설정
```python
config = {
    'algorithm': 'fixmatch',
    'net': 'wrn_28_2',
    'use_pretrain': False,  # todo: add pretrain
    'pretrain_path': None,

    # optimization configs
    'epoch': 3,
    'num_train_iter': 150,
    'num_eval_iter': 50,
    'optim': 'SGD',
    'lr': 0.03,
    'momentum': 0.9,
    'batch_size': 64,
    'eval_batch_size': 64,

    # dataset configs
    'dataset': 'cifar10',
    'num_labels': 40,
    'num_classes': 10,
    'img_size': 32,
    'crop_ratio': 0.875,
    'data_dir': './data',

    # algorithm specific configs
    'hard_label': True,
    'uratio': 3,
    'ulb_loss_ratio': 1.0,

    # device configs
    'gpu': 0,
    'world_size': 1,
    'distributed': False,
}
config = get_config(config)
```

### 1.2.2 모델을 선언하고 알고리즘의 특정
```python
from semilearn import get_dataset, get_data_loader, get_net_builder, get_algorithm, get_config, Trainer

algorithm = get_algorithm(config,  get_net_builder(config.net, from_name=False), tb_log=None, logger=None)
```
 
### 1.2.3 데이터셋 생성
```python
dataset_dict = get_dataset(config, config.algorithm, config.dataset, config.num_labels, config.num_classes, data_dir=config.data_dir)
train_lb_loader = get_data_loader(config, dataset_dict['train_lb'], config.batch_size)
train_ulb_loader = get_data_loader(config, dataset_dict['train_ulb'], int(config.batch_size * config.uratio))
eval_loader = get_data_loader(config, dataset_dict['eval'], config.eval_batch_size)
```

### 1.2.4 학습
```python
trainer = Trainer(config, algorithm)
trainer.fit(train_lb_loader, train_ulb_loader, eval_loader)
```

### 1.2.5 평가 및 prediction활용
```python
trainer.evaluate(eval_loader)

y_pred, y_logits = trainer.predict(eval_loader)
```

# 2. 데이터 셋
MixMatch, ReMixMatch, FixMatch, FlexMatch, SimMatch 순으로 수행되며, 최대한 hyperparameter는 통일하였으며, 각각의 구체적인 parameter는 config 폴더를 참고하면 된다.

intro에서 설명한 대로 image domain에서의 본 방법론들의 성능은 이미 충분히 실험이 되고 분석이 수행되었기 때문에, 이번 실습에서는 특별히 NLP domain의 데이터를 활용해 실험을 수행하였다.  
실험에서 활용한 데이터는 Amazon Review로 text classification에 활용되는 데이터이다.  

각각 방법론에 대한 차이는 SSL 알고리즘만으로 통제하기 위해 두 데이터에 대한 다섯 가지 알고리즘 모두에서 사전 학습된 bert_base_uncased 모델을 통해 tokenizing을 수행하였다.

- Amazon Review
    - class: 5개 
    - 전체 data: train-250000, dev-25000, test-650000
<!-- - Yahoo Answers
    - class: 10개
    - 전체 data: train-500000, dev-50000, test-60000 -->

추가로 각각의 방법론이 label data의 수에 따라 어떤 영향을 받는 지를 확인하기 위해 각각의 train data에서 labeled data의 수를 Amazon: (250, 1000)개 만큼 서로 다른 dataset을 만들어 실험 하였다.

다양한 SSL 방법론 들은 data augmentation을 사용하는데, 본 데이터에서는 사전에 데이터 증강을 수행 해 놓은 결과를 데이터 셋 자체에 포함시켜 두었다.  
그 예시의 형태는 다음과 같다.

```python
    "0": {
        "ori": "THIS is MUSIC at its BESTRob Dougan has done it. He's crafted musical perfection, or close to it anyway. I have finally found the music I've been waiting for my whole life in this album~~",
        "aug_0": "THIS is MUSIC kurzfristig ist BestRob Dougan has done it all. He's superbly musical, or close to it. On that album - Rob D, are you a genius - I finally got the m~~",
        "aug_1": "THIS is MUSIC at its - Um Rob Dougan has deed it. He's the crazy music definition, or close to it anyway. I find the station music I've waited my e~~",
        "label": "4" }
```

모델에서는 random으로 augmented data를 sampling하는 것으로 Stochastic data augmentation을 수행한다. 본 라이브러리 에서는 Amazon Review 각각 마다 두 가지의 augmentated data가 제공되어 최대 Augmentation의 수가 2로 제한된 상황이다. 
<!-- , Yahoo: (500, 2000) -->

실험의 설계는 5개 model에 대해 Amazon Review의 labeled data가 250, 1000개 일때를 모두 실험 하므로, 총 5 x 2 = 10회의 실험이 수행된다. Titan RTX 3개에 병렬화 하여 multiprocessing_distributed 한 경우에도 실험 시간이 너무 길게 소요되어 현실적으로 학습 data는 50000개로 sampling 하였다.

다음은 sampling에 사용한 코드 이다.
   ```python
    import json
    import random
    from tqdm import tqdm

    with open("train.json", "r") as st_json:
        train_all = json.load(st_json)
        
    idx = random.sample(range(1,len(train_all)+1),50000)
    idx =list(map(str, idx))
    new_train = {}

    new_key=0
    for key, value in tqdm(train_all.items()): 
        if key in idx:
            new_train[new_key] = value
            new_key+=1

    with open("new_tran.json", 'w') as outfile:
        json.dump(new_train, outfile)
   ```
결과적으로 다음과 같은 비율로 5개 class가 sampling 되었으며 [0.19846, 0.2022, 0.19914, 0.19932, 0.20088],  
실제 model에 사용될 때는 oversampling하여 동일한 비율로 통일되었다.


# 3. 실험 step by step

1. 라이브러리에서 사용하는 python 버전이 3.8이므로 conda 혹은 docker container를 활용하여 해당 환경을 구성한다.
    ```python
    conda create --name usb python=3.8
    ```

2. repository clone 
   ```python
   git clone https://github.com/microsoft/Semi-supervised-learning.git
   ```
3. install required packages
   ```python
   pip install -r requirements.txt
   pip install semilearn
   ```
4. 데이터 다운로드 https://github.com/microsoft/Semi-supervised-learning/tree/main/preprocess
   ```python
   cd ../
   mkdir data & cd data
   wget https://wjdcloud.blob.core.windows.net/dataset/usbdata.tar.gz
   tar -xvf usbdata.tar.gz
   ```
5. 실험의 목적에 맞는 config 파일을 만들기
   본 튜토리얼 repository의 config 폴더 참고
    <p align="center">   <image src="./images/2022-12-27-17-16-53.png" width="40%"/>   </p>

6. 학습 수행
   ```python
   python train.py --c config/fixmatch_amazon_review_200_0.yaml
   ```

7. 평가
   ```python
   python eval.py --dataset amazon_review_200 --num_classes 10 --load_path /PATH/TO/CHECKPOINT
   ```

# 4. 실험 설정 (config)

## 4.1 공통 설정
- 병렬화: 한번의 batch size는 9개로 설정하고 3개의 gpu에서 각각 3개씩을 처리하도록 병렬화
- 자연어 처리 backbone model: bert_base_uncased, max_length: 512
 <p align="center">   <image src="./images/2022-12-27-21-37-59.png" width="40%"/>   </p>


## 4.2 각 모델별 설정
- MixMatch
   ```YAML
   mixup_alpha: 0.5
   T: 0.5
   ulb_loss_ratio: 100
   unsup_warm_up: 0.4
   mixup_manifold: True
   ```

- ReMixMatch
   ```YAML
   mixup_alpha: 0.75
   T: 0.5
   kl_loss_ratio: 0.5
   ulb_loss_ratio: 1.5
   rot_loss_ratio: 0.0
   unsup_warm_up: 0.015625
   mixup_manifold: True
   ```
- FixMatch
   ```YAML
   hard_label: True
   T: 0.5
   p_cutoff: 0.95
   ulb_loss_ratio: 1.0
   ```
- FlexMatch
   ```YAML
   hard_label: True
   T: 0.5
   p_cutoff: 0.95
   ulb_loss_ratio: 1.0
   thresh_warmup: True
   ```
- SimMatch
   ```YAML
   T: 0.1
   p_cutoff: 0.95
   ulb_loss_ratio: 1.0
   in_loss_ratio: 1.0
   proj_size: 128
   K: 256
   da_len: 32
   smoothing_alpha: 0.9
   ```

# 5. 실험 결과

## 5.1 50000개의 데이터 중 250개만 labeled 인경우
eval dataset의 accuracy 기준으로 성능을 정리한 것이다. 첫 행의 Only labeled의 경우는 비교를 위해 250개의 데이터 만으로 학습을 수행 했을 때의 성능을 나타낸다.

| model    | amazon_review_250       |
| ---------- | --------- |
| Only labeled | 0.35789  |
| MixMatch  | 0.4014 |
| ReMixMatch  | 0.2001 |
| FixMatch  | 0.49324 |
| FlexMatch  | 0.53192 |
| SimMatch  | 0.49912 |
* ReMixMatch의 경우 학습 과정에서 오류 발생

기본적으로 단지 250개의 labeled 데이터 만을 사용했을 때보다는 성능이 향상되는 경향을 보이고 있고 FlexMatch의 경우가 가장 높은 성능을 보이고 있다. 그러나 각 방법론 사이에 성능 차이가 유의하지는 않는 것으로 보인다.  
추가로 직접 실험한 환경 이외에 repository에서 제공하는 실험 결과를 첨부하면 다음과 같다. 

| model    | amazon_review_250       |
| ---------- | -------------- |
| Only labeled | 36.81±0.05 |
| MixMatch  | 63.42±1.34 |
| ReMixMatch  | 80.0±0.0 |
| FixMatch  | 47.85±1.22|
| FlexMatch  |  45.75±1.21 |
| SimMatch  | 47.27±1.73 |

MixMatch와 ReMixMatch의 성능은 대폭 증가되고, FixMatch, FlexMatch, SimMatch의 성능은 소폭 감소하였다. 이러한 결과는 우선 사용한 데이터의 수에서 차이가 나며, 세부적인 하이퍼파라미터의 차이에 의한 것일 것이다.  
현재의 결과만 보면 MixMatch와 ReMixMatch의 경우가 조금 더 hyperparameter에 민감하다고 관찰이된다.

## 5.2 50000개의 데이터 중 1000개만 labeled 인경우

* 실험결과

| model    | amazon_review_1000       |
| ---------- | -------------- |
| Only labeled | 0.3726 |
| MixMatch  | 0.4058 |
| ReMixMatch  | 0.2001 |
| FixMatch  | 0.5414|
| FlexMatch  |  0.55656 |
| SimMatch  | 0.5424 |

labeled data의 수가 증가한 만큼 labeled data만 사용한 경우의 성능이 약간 증가였다. 그 증가 폭에 비하면 FixMatch, FlexMatch, SimMatch의 성능 증가 폭이 유의미한 것으로 평가된다. 

* Reporting

| model    | amazon_review_1000       |
| ---------- | -------------- |
| Only labeled | 36.88±0.19 |
| MixMatch  | 69.97±7.09 |
| ReMixMatch  | 80.0±0.0 |
| FixMatch  | 43.73±0.45|
| FlexMatch  |  43.14±0.82 |
| SimMatch  | 43.09±0.5 |

위 결과는 repository의 결과인데, 예상과는 다르게 labeled data의 수가 4배나 증가했음에도 큰 성능의 향상이 관찰되지 않는다. 유일하게 성능이 향상된 모델은 MixMatch로 본 모델이 50000개 중 1000개 비율, 즉, 0.02의 labeled 비율에 대해 성능이 향상된다는 것을 알 수 있다. 의외인 것은 FixMatch, FlexMatch, SimMatch의 경우 성능이 감소했단 것인데, 나머지 파라미터가 모두 동일했음에도 이러한 결과를 얻은 것은 의아하게 느껴진다. 이 같은 결과에 대한 원인을 추론해 보자면, 현재 사용하고 있는 데이터가 nlp text data로 본 방법론들이 제안된 image domain과 다른 성질을 가지기 때문으로 보인다. 현재는 모든 data를 bert를 통해 전처리 하는데, 이미 이 과정에서 각 데이터의 특징이 충분하게 추출되지 못하여 데이터의 증가가 확실한 정보의 증가로 이어지지 못할 수도 있다고 생각이된다. 

